In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,6116
2,Huelva,Confirmados PDIA 14 días,2501
3,Huelva,Tasa PDIA 14 días,"489,6787621171509"
4,Huelva,Confirmados PDIA 7 días,1060
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,91
629,Municipio de Huelva sin especificar,Total Confirmados,193
630,Municipio de Huelva sin especificar,Curados,20


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  6116.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  1621.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 519 personas en los últimos 7 días 

Un positivo PCR cada 219 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,6116.0,1060.0,2501.0,510743.0,207.540779,489.678762,599.0
Huelva-Costa,3089.0,546.0,1251.0,288115.0,189.507662,434.201621,348.0
Huelva (capital),1621.0,277.0,656.0,143663.0,192.812346,456.624183,168.0
Condado-Campiña,1907.0,255.0,759.0,155057.0,164.455652,489.497411,134.0
Lepe,478.0,84.0,210.0,27431.0,306.222887,765.557216,60.0
Sierra de Huelva-Andévalo Central,928.0,168.0,358.0,67571.0,248.627370,529.813085,51.0
Cartaya,152.0,36.0,61.0,19974.0,180.234305,305.397016,28.0
Palma del Condado (La),281.0,37.0,113.0,10761.0,343.834216,1050.088282,26.0
Aracena,225.0,74.0,137.0,8107.0,912.791415,1689.897619,25.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Escacena del Campo,84.0,8.0,42.0,2284.0,350.262697,1838.879159,4.0
Aracena,225.0,74.0,137.0,8107.0,912.791415,1689.897619,25.0
Gibraleón,266.0,60.0,157.0,12607.0,475.926073,1245.339891,16.0
Palma del Condado (La),281.0,37.0,113.0,10761.0,343.834216,1050.088282,26.0
Villarrasa,68.0,2.0,19.0,2176.0,91.911765,873.161765,2.0
Manzanilla,41.0,7.0,18.0,2135.0,327.868852,843.091335,4.0
Lepe,478.0,84.0,210.0,27431.0,306.222887,765.557216,60.0
Villalba del Alcor,75.0,6.0,24.0,3338.0,179.748352,718.993409,2.0
Valverde del Camino,187.0,36.0,90.0,12820.0,280.811232,702.028081,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villarrasa,68.0,2.0,19.0,2176.0,91.911765,873.161765,2.0,0.105263
Almonaster la Real,29.0,1.0,9.0,1819.0,54.975261,494.777350,1.0,0.111111
Trigueros,97.0,6.0,40.0,7713.0,77.790743,518.604953,5.0,0.150000
Escacena del Campo,84.0,8.0,42.0,2284.0,350.262697,1838.879159,4.0,0.190476
Zalamea la Real,36.0,2.0,8.0,3068.0,65.189048,260.756193,0.0,0.250000
Villalba del Alcor,75.0,6.0,24.0,3338.0,179.748352,718.993409,2.0,0.250000
Paterna del Campo,40.0,5.0,19.0,3509.0,142.490738,541.464805,3.0,0.263158
Niebla,57.0,7.0,26.0,4117.0,170.026718,631.527812,3.0,0.269231
Beas,23.0,3.0,10.0,4257.0,70.472163,234.907212,2.0,0.300000
